# Анализ базы данных сервиса для чтения книг по подписке

## Введение

В данном проекте будет проанализирована база данных сервиса для чтения книг по подписке. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

Эти данные помогут сформулировать ценностное предложение для нового продукта.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine 

In [2]:
# устанавливаем параметры
db_config = {'user': 'placeholder', # имя пользователя
'pwd': 'placeholder', # пароль
'host': 'placeholder',
'port': placeholder, # порт подключения
'db': 'placeholder'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

### Описание данных

Исследуем таблицы. Для этого выведем первые строки.

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [3]:
query = ''' 
SELECT *
FROM books
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


In [31]:
query = ''' 
SELECT COUNT(book_id)
FROM books
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,1000


Всего в базе 1000 книг.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [4]:
query = ''' 
SELECT *
FROM authors
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

In [5]:
query = ''' 
SELECT *
FROM publishers
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [6]:
query = ''' 
SELECT *
FROM ratings
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [7]:
query = ''' 
SELECT *
FROM reviews
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


In [8]:
query = ''' 
SELECT COUNT(review_id)
FROM reviews
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,2793


Всего отзывов 2793.

**Вывод.** Данные соответствуют описанию.

## Исследование

Посчитаем, сколько книг вышло после 1 января 2000 года.

In [9]:
query = ''' 
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000/01/01'
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


После 1 января 2000 года вышло 819 книг.

Для каждой книги посчитаем количество обзоров и среднюю оценку.

In [51]:
query = ''' 
SELECT b.title,
       COUNT(DISTINCT(re.review_id)) count_reviews,
       AVG(ra.rating) avg_rating
FROM books b
JOIN ratings ra ON b.book_id=ra.book_id
LEFT JOIN reviews re ON b.book_id=re.book_id
GROUP BY b.book_id
'''

pd.io.sql.read_sql(query, con = engine)

,title,count_reviews,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


In [49]:
query = ''' 
WITH a AS (SELECT b.title,
       COUNT(DISTINCT(re.review_id)) count_reviews,
       AVG(ra.rating) avg_rating
FROM books b
JOIN ratings ra ON b.book_id=ra.book_id
LEFT JOIN reviews re ON b.book_id=re.book_id
GROUP BY b.book_id)

SELECT SUM(count_reviews)
FROM a
'''

pd.io.sql.read_sql(query, con = engine)

,sum
0,2793.0


Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.

In [11]:
query = ''' 
SELECT p.publisher,
       COUNT(b.book_id) books
FROM publishers p
JOIN books b ON p.publisher_id=b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,publisher,books
0,Penguin Books,42


Таким издательством стало Penguin Books с 42 книгами.

Определим автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками.

In [12]:
query = ''' 
WITH num_ratings AS (SELECT book_id
                    FROM ratings
                    GROUP BY book_id
                    HAVING COUNT(rating_id) > 50)

SELECT a.author,
       AVG(r.rating) avg_rating
FROM authors a
JOIN books b ON a.author_id=b.author_id
JOIN ratings r ON b.book_id=r.book_id
WHERE b.book_id IN (SELECT book_id
                    FROM num_ratings)
GROUP BY a.author_id
ORDER BY AVG(r.rating) DESC
LIMIT 1
'''

pd.io.sql.read_sql(query, con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Самые высокие оценки у книг Джоан Роулинг с иллюстрациями Мэри Гранпрэ.

Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
query = '''
WITH rating_count AS (SELECT username
                      FROM ratings
                      GROUP BY username
                      HAVING COUNT(rating_id) > 50),
     review_count AS (SELECT username,
                             COUNT(review_id) review_cnt
                      FROM reviews
                      WHERE username in (SELECT username
                                         FROM rating_count)
                      GROUP BY username)
                      
SELECT ROUND(AVG(review_cnt), 1) avg_review_cnt
FROM review_count
'''

pd.io.sql.read_sql(query, con = engine)

,avg_review_cnt
0,24.3


Пользователи, которые поставили больше 50 оценок, в среднем пишут 24,3 обзора.

## Вывод
В проекты были выяснены такие факты:
* После 1 января 2000 года вышло 819 книг.
* Издательство Penguin Books выпустило наибольшее число книг толще 50 страниц - 42 книги.
* Самые высокие оценки у книг Джоан Роулинг с иллюстрациями Мэри Гранпрэ.
* Пользователи, которые поставили больше 50 оценок, в среднем пишут 24,3 обзора.

Также были посчитаны количество обзоров и средняя оценка для каждой книги.